Calculate functionals with symbolics 
create kernel 

Create points 

construnct distance tnsors 


apply kernel to a distance tensor 

assemble RHS and matrices 

Assemble DAE 

Solve with diffeq.jl or custom code 





In [1]:
using Revise
using CairoMakie
using Symbolics
using Latexify
using DifferentialEquations
using Sundials
using BenchmarkTools
#cd("..")
include("RBFunctions.jl")
#cd("circular_stokes_flow")
using Plots

In [2]:
mutable struct Sol
    t
    sol
end

In [3]:
function (p::Sol)(x) # linear intrpolation
    if x > p.t[end]
        error("out of bounds")
    end
    dt = p.t[2] - p.t[1]
    n = Int(x÷dt)+1
    rem = x%dt
    return p.sol[:,n]+(p.sol[:,n+1] - p.sol[:,n])*(rem/dt)
end

In [6]:
@variables ϵ r x₁ x₂ t Δt;
const nu = 1.0
#ϕ = 1//945 * ((ϵ*r)^5 +15*(ϵ*r)^3 + 105*(ϵ*r)^2 + 945*(ϵ*r)+ 945)* exp(-ϵ*r)
ϕ = r^7 #exp(-r^2*ϵ^2)
ϕ2 = r^3
ϕ = substitute(ϕ, r=>sqrt(x₁^2 + x₂^2)) 
#display(ϕ)  
Δ(exprs) = expand_derivatives((Differential(x₁)^2)(exprs) + (Differential(x₂)^2)(exprs))
∂₁(exprs) = expand_derivatives(Differential(x₁)(exprs))
∂₂(exprs) = expand_derivatives(Differential(x₂)(exprs))
∂ₜ(exprs) = expand_derivatives(Differential(t)(exprs))

Φ_div = ([-∂₂(∂₂(ϕ)) ∂₁(∂₂(ϕ)) 0.0 ; ∂₁(∂₂(ϕ)) -∂₁(∂₁(ϕ)) 0.0; 0.0 0.0 ϕ])
ΔΦ_div= Δ.([-∂₂(∂₂(ϕ)) ∂₁(∂₂(ϕ)); ∂₁(∂₂(ϕ)) -∂₁(∂₁(ϕ))])
Φ_curl = ([-∂₁(∂₁(ϕ)) -∂₁(∂₂(ϕ)); -∂₁(∂₂(ϕ)) -∂₂(∂₂(ϕ))])
Φ = [ϕ 0.0 0.0; 0.0 ϕ 0.0;0.0 0.0 ϕ]
#ΔΦ = [Δ(ϕ) 0 ; 0 Δ(ϕ)]
true_u₁ = -x₂*π*sin(π*0.5*(x₁*x₁ + x₂*x₂))*sin(π*t)
true_u₂ =  x₁*π*sin(π*0.5*(x₁*x₁ + x₂*x₂))*sin(π*t)
true_∂ₜu₁ = ∂ₜ(true_u₁)
true_∂ₜu₂  = ∂ₜ(true_u₂)
true_p = sin(x₁-x₂+t)
true_∂₁p = ∂₁(true_p)
true_∂₂p = ∂₂(true_p)
f₁ = ∂ₜ(true_u₁) - nu*Δ(true_u₁) + true_∂₁p
f₂ = ∂ₜ(true_u₂) - nu*Δ(true_u₂) + true_∂₂p


true_u₁ = eval(build_function(true_u₁,x₁, x₂, t))
true_u₂ = eval(build_function(true_u₂,x₁, x₂, t))
true_p = eval(build_function(true_p,x₁, x₂, t))
true_∂₁p = eval(build_function(true_∂₁p,x₁, x₂, t))
true_∂₂p = eval(build_function(true_∂₂p,x₁, x₂, t))
f₁ = eval(build_function(f₁,x₁, x₂, t))
f₂ = eval(build_function(f₂,x₁, x₂, t))
∂ₜu₁ = eval(build_function(true_∂ₜu₁,x₁, x₂, t))
∂ₜu₂  = eval(build_function(true_∂ₜu₂,x₁, x₂, t))
zero_func(x₁,x₂,t) = 0.0


λ1y(x) = x[1] - Δt*Δ(x[1]) -  Δt*∂₁(x[3])
λ2y(x) = x[2] - Δt*Δ(x[2]) -  Δt*∂₂(x[3])
λ3y(x) = x[1]
λ4y(x) = x[2]

λ1x(x) = x[1] - Δt*Δ(x[1]) +  Δt*∂₁(x[3])
λ2x(x) = x[2] - Δt*Δ(x[2]) +  Δt*∂₂(x[3])
λ3x(x) = x[1]
λ4x(x) = x[2]

λu(x) = x[1]
λv(x) = x[2]
λp(x) = x[3]


λp (generic function with 1 method)

In [3]:
#generate points for circular flow benchmark 
using DistMesh2D
fdistance(p) = sqrt(sum(p .^ 2)) - 1
fedgelength(p) = 1/(sqrt(sum(p .^ 2))+1)
boundingbox = [-1.1 -1.1; 1.1 1.1]
initdistance = 0.1
x, y = distmesh2d(fdistance, fedgelength, boundingbox, initdistance)

# generate points, reference solution, forcing vector, and boundary conditions 
tx = unique(filter(e -> !isequal(e,NaN) ,x))
ty = unique(filter(e -> !isequal(e,NaN) ,y))

points = hcat(tx,ty)'

Internal_points = []
Boundary_points = []
suma = 0
for i in 1:length(tx)
    if abs(sum(points[:,i] .^2) - 1) < 1e-8
        #println(i)
        suma +=1
        append!(Boundary_points,[points[:,i]])
    else
        append!(Internal_points,[points[:,i]])
    end
    
end

Internal_points = hcat(Internal_points...)
Boundary_points = hcat(Boundary_points...)
All_points = hcat(Internal_points,Boundary_points)
N_i = size(Internal_points)[2]
N_b = size(Boundary_points)[2]
N = N_i + N_b
println("total number of nodes: ",N)
println("number og internal_nodes: ",N_i)

total number of nodes: 358
number og internal_nodes: 294


In [4]:
Plots.scatter(Internal_points[1,:],Internal_points[2,:],aspect_ratio=:equal,background_color = :transparent,foreground_color = :black,
label = "Domain points",ticks = false,showaxis = false)
Plots.scatter!(Boundary_points[1,:],Boundary_points[2,:],aspect_ratio=:equal,background_color = :transparent,foreground_color = :black,
label ="Boundary points", ticks = false,showaxis = false)
savefig("internal_domain_points.png")

In [8]:
parameter = 4
K = construct_kernel_array(Φ_div,[λ1x,λ2x,λ3x,λ4x],[λ1y,λ2y,λ3y,λ4y])
#display(build_function(K[1,1],[x₁,x₂],ϵ))
K = substitute.(K, Δt=>0.001)
display(K)
K = compile_kernel_array(K)

R = construct_kernel_array(Φ_div,[λu,λv,λp],[λ1y,λ2y,λ3y,λ4y])
#display(R)
#R = substitute.(R, Δt=>0.001) 
R = compile_kernel_array(R)
"""
T1 = crete_block_point_tensors([Internal_points,Internal_points,Boundary_points,Boundary_points],
[Internal_points,Internal_points,Boundary_points,Boundary_points])
#display(typeof(T1))
T2 = crete_block_point_tensors([Internal_points,Internal_points,Internal_points],
[Internal_points,Internal_points,Boundary_points,Boundary_points])

A = generate_block_matrices(K,T1,parameter)
A = flatten(A)

S = generate_block_matrices(R,T2,parameter)
S = flatten(S)

#@btime K[1,1]([1,1],1)
println(cond(A))
#display(S)

g = generate_vector_function([true_u₁,true_u₂],Boundary_points)
f = generate_vector_function([f₁,f₂],Internal_points)
ref_sol = generate_vector_function([true_u₁,true_u₂,true_p],Internal_points)
f(1.2)
"""

4×4 Matrix{Num}:
 -7.0e-6(sqrt(x₁^2 + x₂^2)^5) - 3.5e-5(x₁^2)*(sqrt(x₁^2 + x₂^2)^3) + 0.14(sqrt(x₁^2 + x₂^2)^3) + 0.105(x₂^4)*(sqrt(x₁^2 + x₂^2)^-1) + 0.105(x₁^2)*sqrt(x₁^2 + x₂^2) + 0.735(x₂^2)*sqrt(x₁^2 + x₂^2) + 0.105(x₁^2)*(x₂^2)*(sqrt(x₁^2 + x₂^2)^-1) + 0.00168(x₂^4)*(sqrt(x₁^2 + x₂^2)^-3) + 0.00063(x₁^2)*(x₂^2)*(sqrt(x₁^2 + x₂^2)^-3) - 0.00189sqrt(x₁^2 + x₂^2) - 0.000315(x₁^2)*(sqrt(x₁^2 + x₂^2)^-1) - 0.005355(x₂^2)*(sqrt(x₁^2 + x₂^2)^-1) - 0.000315(x₂^6)*(sqrt(x₁^2 + x₂^2)^-5) - 0.000315(x₁^2)*(x₂^4)*(sqrt(x₁^2 + x₂^2)^-5) + 0.000105(x₁^4)*(sqrt(x₁^2 + x₂^2)^-3) + 0.000105(x₂^4)*(sqrt(x₁^2 + x₂^2)^-3) + 0.00126(x₁^2)*(x₂^2)*(sqrt(x₁^2 + x₂^2)^-3) - 0.00063sqrt(x₁^2 + x₂^2) - 0.000945(x₁^2)*(sqrt(x₁^2 + x₂^2)^-1) - 0.000945(x₂^2)*(sqrt(x₁^2 + x₂^2)^-1) - 0.000315(x₁^4)*(x₂^2)*(sqrt(x₁^2 + x₂^2)^-5) - 0.000315(x₁^2)*(x₂^4)*(sqrt(x₁^2 + x₂^2)^-5) + 0.14(sqrt(x₁^2 + x₂^2)^3) + 0.105(x₂^4)*(sqrt(x₁^2 + x₂^2)^-1) + 0.105(x₁^2)*sqrt(x₁^2 + x₂^2) + 0.735(x₂^2)*sqrt(x₁^2 + x₂^2) + 0.105(

"T1 = crete_block_point_tensors([Internal_points,Internal_points,Boundary_points,Boundary_points],\n[Internal_points,Internal_points,Boundary_points,Boundary_points])\n#display(typeof(T1))\nT2 = crete_block_point_tensors([Internal_points,Internal_points,Internal_points],\n[In" ⋯ 171 bytes ⋯ "= flatten(S)\n\n#@btime K[1,1]([1,1],1)\nprintln(cond(A))\n#display(S)\n\ng = generate_vector_function([true_u₁,true_u₂],Boundary_points)\nf = generate_vector_function([f₁,f₂],Internal_points)\nref_sol = generate_vector_function([true_u₁,true_u₂,true_p],Internal_points)\nf(1.2)\n"

In [47]:

function generate_solution(A,S,u0,dt,N_t,N_i,N_b,f,g)
    n = size(S)[1]
    RHS = zeros(N_i*2+2*N_b)
    RHS[1:2*N_i] .= u0 .+ dt*f(0.0+dt)
    RHS[2*N_i+1:end] .= g(0.0+dt)
    A_inv = inv(A)
    solution_array = zeros((n,N_t+1))
    u = vcat(u0,zeros(N_i))
    solution_array[:,1] .= u
    t_array = [0.0]
    for i in 1:N_t
        append!(t_array,[i*dt])
        u = S*A_inv*RHS
        solution_array[:,i+1] .= u
        # update RHS for next iteration
        RHS[1:2*N_i] .= u[1:2*N_i] .+ dt*f((i+1)*dt)
        RHS[2*N_i+1:end] .= g((i+1)*dt)
    end
    solution = Sol(t_array,solution_array)
    return solution
end
u0 = ref_sol(0.0)[1:2*N_i]
dt = 0.001
N_t = 2000
solution = generate_solution(A,S,u0,dt,N_t,N_i,N_b,f,g)


Sol([0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009000000000000001  …  1.991, 1.992, 1.993, 1.994, 1.995, 1.996, 1.997, 1.998, 1.999, 2.0], [0.0 0.00596480281014167 … -0.00506624000460094 0.0020228961234144555; 0.0 0.007736181993225017 … -0.006411647590320278 0.001749129258060494; … ; 0.0 -0.39347726046397413 … 0.3706239442230765 0.3709424117535731; 0.0 -0.18782948412646816 … 0.35495581348295113 0.3550880280371048])

In [42]:
maximum(solution(0.5)[1:2*N_i] - ref_sol(0.5)[1:2*N_i])
(solution(0.5)[1+2*N_i:end] - ref_sol(0.5)[1+2*N_i:end]) .- (solution(0.5)[1+2*N_i:end] - ref_sol(0.5)[1+2*N_i:end])[1]

294-element Vector{Float64}:
 0.0
 0.04095680519399969
 0.03994272894878448
 0.03631406481646665
 0.03932770288503046
 0.03160185151980549
 0.02753415349109145
 0.022947340200078936
 0.015456004219793262
 0.03389436350653463
 0.030416488400618447
 0.036693588872385385
 0.03388273679009557
 ⋮
 0.0842313556096812
 0.059710168206040026
 0.08389116294598187
 0.09861445243257544
 0.130340411503511
 0.064186648126069
 0.1529166104095613
 0.11537241763236006
 0.07232001694010542
 0.08479980807231091
 0.08124860937904288
 0.10291501142475235

In [48]:
time = Observable(0.0)
xs = Internal_points[1,:]
ys = Internal_points[2,:]
us = @lift solution($time)[1:N_i] #- u_true($time)[1:N_i]
vs = @lift solution($time)[N_i+1:2*N_i]  #- u_true($time)[N_i+1:end] 
title = @lift string(round($time, digits=2))
strength2 = @lift sqrt.(solution($time+0.01)[1:N_i] .^ 2 .+ solution($time+0.01)[N_i+1:2*N_i] .^ 2) 

fig = Figure(resolution = (800, 800))
Axis(fig[1, 1], backgroundcolor = "black",title = title)
arrows!(xs, ys, us, vs, arrowsize = 10, lengthscale = 0.1, arrowcolor = strength2, linecolor = strength2) #, 
framerate = 30
timestamps = range(0.0, 1.999, step=1/framerate)
record(fig, "ODE_method.gif", timestamps; framerate = framerate) do ti
    time[] = ti
end


"ODE_method.gif"

LoadError: syntax: invalid escape sequence